# Modelling - Billboard Hot 100 & Spotify Track Data

## 1.0 Import Data

## 2.0 Refining the Model

In [ ]:
# using the 52 features

# use Bayesian optimization on random forest to pick a better number of features? GridSearchCV was taking too long

# test it out on some unseen data?